In [ ]:
%sql
-- =========================================================
-- SILVER: quality_inspections_clean (dedupe + idempotência via hash)
-- =========================================================
CREATE TABLE IF NOT EXISTS silver.quality_inspections_clean (
  inspection_id       STRING,
  production_order_id  STRING,
  equipment_id           STRING,
  inspection_type     STRING,
  inspection_date     TIMESTAMP,
  inspector_id        STRING,
  passed              BOOLEAN,
  failed_quantity     INT,
  total_quantity      INT,
  defect_codes        STRING,
  notes               STRING,
  last_update         TIMESTAMP,
  row_hash            STRING
) USING DELTA;

CREATE OR REPLACE TEMP VIEW stage_quality_inspections AS
SELECT
  inspection_id,
  production_order_id,
  equipment_id,
  LOWER(TRIM(inspection_type)) AS inspection_type_norm,
  COALESCE(
    try_to_timestamp(inspection_date, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(inspection_date, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(inspection_date, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(inspection_date, 'yyyy-MM-dd'),
    try_to_timestamp(inspection_date, 'yyyy/MM/dd')
  ) AS inspection_date_ts,
  inspector_id,
  CASE
    WHEN LOWER(TRIM(passed)) IN ('true', '1', 'yes', 'y', 't') THEN TRUE
    WHEN LOWER(TRIM(passed)) IN ('false', '0', 'no', 'n', 'f') THEN FALSE
    ELSE NULL
  END AS passed_bool,
  CAST(regexp_replace(failed_quantity, ',', '.') AS INT) AS failed_quantity_norm,
  CAST(regexp_replace(total_quantity, ',', '.') AS INT) AS total_quantity_norm,
  UPPER(TRIM(defect_codes)) AS defect_codes_norm,
  notes,
  COALESCE(
    try_to_timestamp(last_update, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(last_update, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(last_update, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(last_update, 'yyyy-MM-dd'),
    try_to_timestamp(last_update, 'yyyy/MM/dd')
  ) AS last_update_ts
FROM bronze.quality_inspections
WHERE inspection_id IS NOT NULL;

CREATE OR REPLACE TEMP VIEW stage_quality_inspections_win AS
SELECT *
FROM stage_quality_inspections
WHERE last_update_ts >= date_sub(current_timestamp(), 60);

CREATE OR REPLACE TEMP VIEW stage_quality_inspections_dedup AS
SELECT
  inspection_id,
  production_order_id,
  equipment_id,
  inspection_type_norm AS inspection_type,
  inspection_date_ts AS inspection_date,
  inspector_id,
  passed_bool AS passed,
  failed_quantity_norm AS failed_quantity,
  total_quantity_norm AS total_quantity,
  defect_codes_norm AS defect_codes,
  notes,
  last_update_ts AS last_update
FROM (
  SELECT
    s.*,
    ROW_NUMBER() OVER (
      PARTITION BY inspection_id
      ORDER BY last_update_ts DESC NULLS LAST,
               inspection_id DESC
    ) AS rn
  FROM stage_quality_inspections_win s
  WHERE last_update_ts IS NOT NULL
) z
WHERE rn = 1;

CREATE OR REPLACE TEMP VIEW stage_quality_inspections_final AS
SELECT
  inspection_id,
  production_order_id,
  equipment_id,
  inspection_type,
  inspection_date,
  inspector_id,
  passed,
  failed_quantity,
  total_quantity,
  defect_codes,
  notes,
  last_update,
  sha2(concat_ws('||',
    coalesce(production_order_id,''),
    coalesce(equipment_id,''),
    coalesce(inspection_type,''),
    coalesce(date_format(inspection_date,'yyyy-MM-dd HH:mm:ss'),''),
    coalesce(inspector_id,''),
    cast(coalesce(passed,false) as string),
    cast(coalesce(failed_quantity,0) as string),
    cast(coalesce(total_quantity,0) as string),
    coalesce(defect_codes,'')
  ), 256) AS row_hash
FROM stage_quality_inspections_dedup;

MERGE INTO silver.quality_inspections_clean AS t
USING stage_quality_inspections_final AS s
ON t.inspection_id = s.inspection_id
WHEN MATCHED AND (t.row_hash IS NULL OR t.row_hash <> s.row_hash) THEN UPDATE SET
  t.production_order_id = s.production_order_id,
  t.equipment_id        = s.equipment_id,
  t.inspection_type     = s.inspection_type,
  t.inspection_date     = s.inspection_date,
  t.inspector_id        = s.inspector_id,
  t.passed              = s.passed,
  t.failed_quantity     = s.failed_quantity,
  t.total_quantity      = s.total_quantity,
  t.defect_codes        = s.defect_codes,
  t.notes               = s.notes,
  t.last_update         = s.last_update,
  t.row_hash            = s.row_hash
WHEN NOT MATCHED THEN INSERT (
  inspection_id, production_order_id, equipment_id, inspection_type, inspection_date,
  inspector_id, passed, failed_quantity, total_quantity, defect_codes, notes, last_update, row_hash
) VALUES (
  s.inspection_id, s.production_order_id, s.equipment_id, s.inspection_type, s.inspection_date,
  s.inspector_id, s.passed, s.failed_quantity, s.total_quantity, s.defect_codes, s.notes, s.last_update, s.row_hash
);
